[View in Colaboratory](https://colab.research.google.com/github/ahracho/TIL/blob/master/Fluect_Python/09_Pythonic_Object.ipynb)

## 9. 파이썬스러운 객체

파이썬 데이터 모델 덕분에 사용자가 정의한 자료형도 내장 자료형과 마찬가지고 자연스럽게 동작할 수 있다. 상속하지 않고도 덕타이핑 메커니즘을 통해 이 모든 것이 가능하다. 단지 객체에 필요한 메서드를 구현하면 기대한 대로 동작한다. 

이번 장에서는 파이썬 객체에서 흔히 볼 수 있는 여러가지 특별 메서드를 구현하는 방법을 보여준다.
- repr(), bytes() 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성사 구현
- format() 내장 함수와 str.format() 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- slots를 이용해서 메모리 절약하기


### 9.1. 객체 표현

- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환
- str() : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환

### 9.2. 벡터 클래스의 부활



In [0]:
from array import array
import math

class Vector2d:
  typecode = 'd'
  
  def __init__(self, x, y):
    self.x = float(x)
    self.y = float(y)
    
  def __iter__(self):
    return (i for i in (self.x, self.y))
  
  def __repr__(self):
    class_name = type(self).__name__
    return "{}({!r}, {!r})".format(class_name, *self)
  
  def __str__(self):
    return str(tuple(self))
  
  def __bytes__(self):
    return (bytes([ord(self.typecode)])+bytes(array(self.typecode, self)))
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.hypot(self.x, self.y)
  
  def __bool__(self):
    return bool(abs(self))
   

### 9.3. 대안 생성자

Vector2d를 bytes로 변환하는 메서드가 있으니, bytes를 Vector2d로 변환하는 메서드도 있어야 한다.  

In [0]:
@classmethod
def frombytes(cls, octets):
  typecode = chr(octets[0])
  memv = memoryview(octets[1:].cast(typecode))
  return cls(*memv)

True
True
False


### 9.4. @classmethod와 @staticmethod

@classmethod 데커레이터가 붙으면 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의한다. @classmethod는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫번째 인수로 받게 만들고 대안 생성자를 구현하기 위해 주로 사용된다. 

@staticmethod 데커레이터는 메서드가 특별한 첫번째 인수를 받지 않도록 메서드를 변경한다. 본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의된 평범한 함수일 뿐이다. 


@classmethod는 쓰임새가 많지만, @staticmethod는 잘 모르겠다. 클래스와 함께 작동하지 않는 함수를 정의하려면, 함수를 모듈에 정의하면 된다.

In [1]:
class Demo:
  @classmethod
  def klassmeth(*args):
    return args
  @staticmethod
  def statmeth(*args):
    return args
  
  
print(Demo.klassmeth()) # (<class '__main__.Demo'>,) 를 첫번째 인수로 받는다
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


### 9.5. 포맷된 출력


In [2]:
br1 = 1/2.43
print(br1)

print(format(br1, '0.4f'))
print('1 BRL = {rate:0.2f} USD'.format(rate=br1))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


In [3]:
# 몇몇 내장 자료형은 포맷 명시 간이 언어에 자신만의 고유한 표현 코드를 가지고 있다. 

print(format(42, 'b')) # int는 b와 x 지원
print(format(2/3, '.1%')) # float은 f와 % 지원


101010
66.7%


In [4]:
# 각 클래스가 format_spec 인수를 자신이 원하는 대로 해석에서 확장할 수 있다.
from datetime import datetime

now = datetime.now()
print(format(now, "%H:%M:%S")) # 24시간 표현

"It's now {:%I:%M %p}".format(now) # 12시간 표현

04:21:08


"It's now 04:21 AM"

클래스에서 __format__() 메서드를 정의하지 않으면, object에서 상속받은 메서드가 str()를 반환한다. 하지만 __format__()을 직접 호출하면 에러가 발생한다. 

In [0]:
def __format__(self, fmt_spec=''):
  if fmt_spce.endswith('p'):
    fmt_spec = fmt_spec[:-1]
    coords = (abs(self), self.angle())
    outer_fmt = '<{}, {}>'
  else:
    coords = self
    outer_fmt = '({}, {})'
    
  components = (format(c, fmt_spec) for c in coords)
  return outer_fmt.format(*components)

### 9.6. 해시 가능한 Vector2d

Vector2d를 해시 가능하게 만들려면 __hash__() 메서드를 구현해야 한다. 또한, 해시 가능한 불변형의 속성을 가져야 한다. 따라서 x와 y를 읽기 전용 속성으로 만들어보자.

In [0]:
class Vector2d:
  typecode = 'd'
  
  def __init__(self, x, y):
    self.__x = float(x)
    self.__y = float(y)
    
  @property
  def x(self):
    return self.__x
  
  @property
  def y(self):
    return self.__y
    
  def __iter__(self):
    return (i for i in (self.x, self.y))
  
  
  def __hash__(self):
    return hash(self.x)^hash(self.y)
  

### 9.7. 파이썬에서의 비공개 속성과 보호된 속성

private 수정자가 없어 비공개 변수를 생성할 수 있는 방법은 없지만, 서브클래스에서 '비공개' 성격의 속성을 실수로 변경하지 못하게 하는 간단한 메커니즘이 있다. 

속성명을 \_\_mood처럼 두개의 언더바로 시작하고 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 파이썬은 언더바와 클래스명을 변수명 앞에 붙여서 객체의 dict에 저장한다. \_\_modd는 \_Dog\_\_mood가 된다. 이러한 파이썬 언어 기능을 이름장식(Name Mangling)이라고 한다.

이름 장식은 안전을 제공하지만, 보안 기능은 아니다. 

### 9.8. __slots__ 클래스 속성으로 공간 절약하기

속성이 몇개 없는 수백만 개의 객체를 다룬다면, __slots__ 클래스 속성을 이용해서 메모리 사용량을 엄청나게 줄일 수 있다. __slots__ 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다. 
> 슈퍼클래스에서 상속받은 __slots__속성은 서브클래스에 영향을 미치지 않고 각 클래스에서 개별적으로 정의된 속성만 고려한다.  



In [0]:
class Vector2d:
  __slots__ = ('__x', '__y')
  typecode = 'd'

#### 9.8.1. __slots__ 사용시 주의할 점
- 인터프리터는 상속된 __slots__을 무시하므로 각 클래스마다 정의해야 한다.
- __dict__를 __slots__에 추가하지 않는 한, 객체는 __slots__에 나열된 속성만 가질 수 있다. 
- __weakref__를 __slots__에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.